In [6]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [3]:
import os

path="Images\Images"
imgs = []
valid_images = [".jpg",".gif",".png",".tga"]
for f in os.listdir(path):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
        
    imgs.append(Image.open(os.path.join(path,f)))
print(len(imgs))

5


In [2]:
from PIL import Image, ImageDraw
from torchvision.transforms import functional as func
import torch
import torchvision


In [4]:

# Run the model on GPU if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load a pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)
model.eval()

print("Model ready")


Model ready


In [8]:

# Choose a random subset of 100 samples to add predictions to
# predictions_view = dataset.take(100, seed=51)

# Get class list
# classes = dataset.default_classes

# Add predictions to samples
for img in imgs:
    # Load image
    image = func.to_tensor(img).to(device)
    c, h, w = image.shape
#     img.show()
    # Perform inference
    preds = model([image])[0]
    labels = preds["labels"].cpu().detach().numpy()
    scores = preds["scores"].cpu().detach().numpy()
    boxes = preds["boxes"].cpu().detach().numpy()

    # Convert detections to FiftyOne format
    detections = []
    
    img1 = ImageDraw.Draw(img)  
    for label, score, box in zip(labels, scores, boxes):
        # Convert to [top-left-x, top-left-y, width, height]
        # in relative coordinates in [0, 1] x [0, 1]
        if score>= .9 :
            x1, y1, x2, y2 = box
#             rel_box = [x1 / w, y1 / h, (x2r - x1) / w, (y2 - y1) / h]
            rel_box=[x1, y1, x2, y2]
            name=label if len(COCO_INSTANCE_CATEGORY_NAMES)<label else COCO_INSTANCE_CATEGORY_NAMES[label]
            print("{} : {}".format( name,rel_box))
            
            img1.rectangle(rel_box,  outline ="blue")
            
    img.show()
    

car : [23.652508, 60.743504, 257.53558, 155.30977]
car : [858.0438, 998.441, 976.85394, 1074.8555]
traffic light : [391.56683, 773.9495, 422.01013, 835.6537]
car : [646.83203, 977.9677, 695.9789, 1006.16486]
traffic light : [787.666, 834.0171, 814.4723, 876.2829]
traffic light : [298.33444, 777.22516, 321.56952, 832.13403]
car : [814.7062, 988.26825, 864.9874, 1030.57]
car : [518.4444, 974.2075, 563.2122, 1001.3399]
person : [790.8404, 474.8026, 1105.1324, 1282.3628]
person : [1302.3429, 491.28067, 1421.5271, 806.267]
person : [234.18555, 528.98254, 351.97476, 736.20593]
chair : [1560.9095, 814.6228, 1765.3408, 1161.5782]
car : [392.82736, 511.21063, 858.8147, 724.14294]
motorcycle : [1.4788136, 658.27423, 222.14511, 816.0829]
car : [975.0077, 522.4594, 1077.8353, 615.6803]
person : [1437.1013, 600.15845, 1603.2349, 904.2796]
car : [1238.5247, 511.14114, 1292.4199, 557.7789]
car : [1023.14215, 487.5967, 1186.3481, 599.8272]
person : [1467.5942, 547.0305, 1537.7488, 670.0198]
car : [121

In [44]:
import pickle


In [45]:

# Save model
with open('mlmodel.pickle', 'wb') as f:
    pickle.dump(model, f)